# =========================
# PRODUCT DATA Analysis
# =========================

## Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = r"/Users/vini/Downloads/farfetch hackathon data/fashion_sales.xlsm"

## load product dataset
df_products = pd.read_excel(file_path, sheet_name="Product_Master")

## load mock dataset
df_mock = pd.read_excel(file_path, sheet_name="MockData")

## Quick check
print("Products dataset:")
print(df_products.head())
print(df_products.info())

print("\nMock dataset:")
print(df_mock.head())
print(df_mock.info())

# distribution of price range per brand (static attribute)

## Defensive checks / convert Price to numeric
print("Before conversion:", df_products['Price'].dtype)
df_products['Price'] = pd.to_numeric(df_products['Price'], errors='coerce')   # convert; invalid -> NaN
print("After conversion:", df_products['Price'].dtype)
print("Price nulls:", df_products['Price'].isna().sum())
print(df_products['Price'].describe())

## If there are NaNs created by conversion, you can decide to drop or fill (here we drop for safety)
df_products = df_products.dropna(subset=['Price']).copy()

## Define broad price bins (use np.inf for open top)
price_bins = [0, 500, 2000, 5000, np.inf]   # edges -> 4 bins
price_labels = ['<500', '500-2000', '2000-5000', '5000+']

## Validate labels length
assert len(price_labels) == len(price_bins) - 1, "Labels length must equal len(bins)-1"

## Create the categorical column
df_products['BroadPriceRange'] = pd.cut(
    df_products['Price'],
    bins=price_bins,
    labels=price_labels,
    include_lowest=True,   # include price == 0 in first bin
    right=False            # left-inclusive, right-exclusive intervals [a, b)
)

## Bar plot (small, readable)
plt.figure(figsize=(6,4))
brands_per_range.plot(kind='bar')
plt.title("Unique Brands per Broad Price Range (Products)")
plt.ylabel("Number of unique brands")
plt.xlabel("Price Range")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## Optional: print the actual brand lists for each range (useful for report)
for pr in price_labels:
    brands_in_range = sorted(df_products.loc[df_products['BroadPriceRange'] == pr, 'Brand'].unique())
    print(f"\nPrice Range {pr} → {len(brands_in_range)} brands")
    print(brands_in_range[:50])   # show first 50 to avoid huge output




## Loop through each price range
for price_range in df_products['BroadPriceRange'].unique():
    # Filter products in this range
    subset = df_products[df_products['BroadPriceRange'] == price_range]

    # Count products per brand and get top 10
    top_brands = subset['Brand'].value_counts().head(10)

    if not top_brands.empty:  # Only plot if brands exist
        plt.figure(figsize=(10,5))
        top_brands.plot(kind='bar', color='skyblue', edgecolor='black')
        plt.title(f"Top 10 Brands in Price Range: {price_range}")
        plt.xlabel("Brand")
        plt.ylabel("Number of Products")
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()



## Distribution of fabrics, finishes, prints


### ---- FABRIC Distribution ----
fabric_counts = df_products['Fabric'].value_counts().head(15)  # top 15 only for clarity
plt.figure(figsize=(10,6))
sns.barplot(y=fabric_counts.index, x=fabric_counts.values, palette="viridis")
plt.title("Top 15 Fabrics in Products Dataset")
plt.xlabel("Count")
plt.ylabel("Fabric")
plt.show()

### ---- FINISH Distribution ----
finish_counts = df_products['Finish'].value_counts().head(15)
plt.figure(figsize=(10,6))
sns.barplot(y=finish_counts.index, x=finish_counts.values, palette="magma")
plt.title("Top 15 Finishes in Products Dataset")
plt.xlabel("Count")
plt.ylabel("Finish")
plt.show()

### ---- PRINT Distribution ----
print_counts = df_products['Print'].value_counts().head(15)
plt.figure(figsize=(10,6))
sns.barplot(y=print_counts.index, x=print_counts.values, palette="cubehelix")
plt.title("Top 15 Prints in Products Dataset")
plt.xlabel("Count")
plt.ylabel("Print")
plt.show()

### count of styles per category
style_per_category = df_products.groupby("Category")['Style'].nunique().sort_values(ascending=False)

plt.figure(figsize=(10,6))
sns.barplot(x=style_per_category.values, y=style_per_category.index, palette="coolwarm")
plt.title("Unique Styles per Category")
plt.xlabel("Number of Unique Styles")
plt.ylabel("Category")
plt.show()

### outlier detection in price
plt.figure(figsize=(10,6))
sns.boxplot(x=df_products['Price'], palette="Set2")
plt.title("Outlier Detection in Product Prices")
plt.xlabel("Price")
plt.show()

### Get actual outlier rows (using IQR method)
Q1 = df_products['Price'].quantile(0.25)
Q3 = df_products['Price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_products[(df_products['Price'] < lower_bound) | (df_products['Price'] > upper_bound)]

print("Number of outliers:", len(outliers))
print(outliers[['Brand','Title','Price']].head(20))  # show first 20



# =========================
# RFM-Like Analysis - Product 
# =========================


import pandas as pd

#### Load your cleaned dataset (Products only) ===

file_path = "/Users/vini/Downloads/farfetch hackathon data/fashion_sales.xlsm"
df_products = pd.read_excel(file_path, sheet_name="Product_Master")   # or CSV if you saved it


print("✅ Dataset loaded:", df_products.shape)
print(df_products.head())

## ===  Recency calculation ===
df_products['Recency'] = df_products['Details'].apply(lambda x: 1 if 'New Season' in str(x) else 0)

## === Frequency calculation ===
frequency = df_products.groupby('Brand')['Product_ID'].count().reset_index()
frequency.rename(columns={'Product_ID': 'Frequency'}, inplace=True)

## === Monetary calculation ===
monetary = df_products.groupby('Brand')['Price'].mean().reset_index()
monetary.rename(columns={'Price': 'Monetary'}, inplace=True)

## ===  Merge RFM table ===
rfm = df_products[['Brand', 'Recency']].drop_duplicates('Brand')
rfm = rfm.merge(frequency, on="Brand", how="left")
rfm = rfm.merge(monetary, on="Brand", how="left")

print("\n📊 RFM Table (first 10 rows):")
print(rfm.head(10))

## === Save result ===
rfm.to_csv("/Users/vini/Downloads/farfetch hackathon data/rfm_table.csv", index=False)
print("\n✅ RFM table saved as rfm_table.csv")

#%%
# --------------------------
#  Business Insights - Product
# --------------------------

### Premium vs Competitive pricing
premium_brands = rfm[rfm['Monetary'] > 3000][['Brand', 'Monetary']].sort_values(by='Monetary', ascending=False)
competitive_brands = rfm[rfm['Monetary'] < 1000][['Brand', 'Monetary']].sort_values(by='Monetary')

print("\n💎 Premium Brands (Avg > $3000):")
print(premium_brands.head(10))

print("\n💲 Competitive Brands (Avg < $1000):")
print(competitive_brands.head(10))

### Most active in FW25 (Recency = 1 + High Frequency)
active_fw25 = rfm[(rfm['Recency'] == 1) & (rfm['Frequency'] > 5)].sort_values(by='Frequency', ascending=False)

print("\n🔥 Active FW25 Brands (High SKUs + New Season):")
print(active_fw25)


### Risk of overstock: many SKUs but low pricing (< $1000)
overstock_risk = rfm[(rfm['Frequency'] > 10) & (rfm['Monetary'] < 1000)]

print("\n⚠️ Overstock Risk Brands (High SKUs + Low Avg Price):")
print(overstock_risk)


### Exclusive positioning: Few SKUs but very high price
exclusive = rfm[(rfm['Frequency'] < 3) & (rfm['Monetary'] > 3000)]

print("\n👑 Exclusive Luxury Brands (Few SKUs + Very High Price):")
print(exclusive)



# =========================
# Mock data EDA
# =========================


###  Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = r"/Users/vini/Downloads/farfetch hackathon data/fashion_sales.xlsm"

### load product dataset
df_mock = pd.read_excel(file_path, sheet_name="MockData")

### Data Overview
print("✅ Dataset Shape:", df_mock.shape)

print("\n📋 Column Names:")
print(df_mock.columns.tolist())

print("\n🔍 Missing Values per Column:")
print(df_mock.isnull().sum())

print("\n🌀 Total Duplicate Rows:", df_mock.duplicated().sum())


### Basic Statistics
print("\n📊 Descriptive Statistics:")
print(df_mock.describe(include="all").T)

print("\n📊 Median values (numeric only):")
print(df_mock.median(numeric_only=True))

print("\n📊 Range of numeric columns:")
for col in df_mock.select_dtypes(include='number').columns:
    col_range = df_mock[col].max() - df_mock[col].min()
    print(f"{col}: {col_range}")


### Data Quality Checks
if "Price" in df_mock.columns:
    print("\n⚠️ Price Values Check:")
    print("Min price:", df_mock["Price"].min(), " | Max price:", df_mock["Price"].max())

if "Quantity" in df_mock.columns:
    print("\n⚠️ Quantity Values Check:")
    print("Min quantity:", df_mock["Quantity"].min(), " | Max quantity:", df_mock["Quantity"].max())


### Outlier detection using IQR
print("\n🚨 Outlier Check (IQR method):")
for col in df_mock.select_dtypes(include='number').columns:
    Q1 = df_mock[col].quantile(0.25)
    Q3 = df_mock[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df_mock[(df_mock[col] < lower_bound) | (df_mock[col] > upper_bound)][col]
    print(f"{col}: {len(outliers)} outliers")

    



# =========================
# Consumer Insight
# =========================

### --- Age Distribution ---

    bins = [0, 25, 40, 55, 75, 100]
    labels = ["Gen Z (<25)", "Millennials (25-40)", "Gen X (40-55)", "Boomers (55-75)", "Older (75+)"]
    df_mock["AgeGroup"] = pd.cut(df_mock["Age"], bins=bins, labels=labels, right=False)
    
    age_dist = df_mock["AgeGroup"].value_counts().sort_index()
    print("\n📊 Age Group Distribution:\n", age_dist)
    
    sns.countplot(data=df_mock, x="AgeGroup", order=labels, palette="pastel")
    plt.title("Age Group Distribution")
    plt.xticks(rotation=30)
    plt.show()

### --- Fashion Influence (Magazines vs Influencers) ---

    magazines = df_mock["Fashion Magazines"].value_counts().head(10)
    influencers = df_mock["Fashion Influencers"].value_counts().head(10)

    print("\n📰 Top Fashion Magazines:\n", magazines)
    print("\n📱 Top Fashion Influencers:\n", influencers)

    fig, axes = plt.subplots(1, 2, figsize=(14,5))
    sns.barplot(x=magazines.values, y=magazines.index, ax=axes[0], palette="muted")
    axes[0].set_title("Top Fashion Magazines Influence")
    sns.barplot(x=influencers.values, y=influencers.index, ax=axes[1], palette="muted")
    axes[1].set_title("Top Fashion Influencers Influence")
    plt.show()

### --- Purchase History ---

    purchase_counts = df_mock["Purchase History"].value_counts()
    print("\n🛍️ Purchase History:\n", purchase_counts)
    
    sns.countplot(data=df_mock, x="Purchase History", palette="coolwarm")
    plt.title("Purchase History Distribution")
    plt.show()

### --- Seasonal Demand ---

    sns.countplot(data=df_mock, x="Season", palette="Set2", order=df_mock["Season"].value_counts().index)
    plt.title("Purchases by Season")
    plt.show()




# =========================
#  Sentiment analysis
# =========================


### import library
from textblob import TextBlob

if "Time Period Highest Purchase" in df_mock.columns:
    sns.countplot(data=df_mock, x="Time Period Highest Purchase", palette="Set3",
                  order=df_mock["Time Period Highest Purchase"].value_counts().index)
    plt.title("Peak Time of Purchases")
    plt.xticks(rotation=30)
    plt.show()

### --- Feedback Analysis (Sentiment from Reviews & Comments) ---
def get_sentiment(text):
    if pd.isna(text) or not isinstance(text, str):
        return "Neutral"
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.1:
        return "Negative"
    else:
        return "Neutral"

for col in ["Customer Reviews", "Social Media Comments"]:
    if col in df_mock.columns:
        df_mock[col + "_Sentiment"] = df_mock[col].apply(get_sentiment)
        sentiment_counts = df_mock[col + "_Sentiment"].value_counts()
        print(f"\n💬 Sentiment Distribution for {col}:\n", sentiment_counts)

        sns.countplot(data=df_mock, x=col + "_Sentiment", order=["Positive", "Neutral", "Negative"], palette="Spectral")
        plt.title(f"Sentiment Analysis of {col}")
        plt.show()

# --------------------------
# RFM Analysis - MOck
# --------------------------

### Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load dataset
file_path = r"/Users/vini/Downloads/farfetch hackathon data/fashion_sales.xlsm"
df_mock = pd.read_excel(file_path, sheet_name="MockData")


# --------------------------
# 1. Handle Recency (Season)
# --------------------------

### Define a seasonal order (latest = lowest number = most recent)
season_order = {
    "New Season": 1,     # Most recent
    "Runway": 2,         # Recent collection
    "Exclusive": 3       # Older/limited availability
}

### Map to numeric recency score
df_mock["Recency"] = df_mock["Season"].map(season_order)

### Fill missing or unknown values with median
df_mock["Recency"].fillna(df_mock["Recency"].median(), inplace=True)


# --------------------------
#  Frequency (Review Count)
# --------------------------
df_mock["Frequency"] = df_mock["Review Count"]


# --------------------------
#  Monetary (Price)
# --------------------------
df_mock["Monetary"] = df_mock["Price"]

# -----------------------------
#  Prepare Recency, Frequency, Monetary
# -----------------------------

### prepare R, F, M...

#### Recency -> Review Count (higher reviews = more recent/engaged)
df_mock["Recency"] = df_mock["Review Count"].fillna(0)

#### Frequency -> Count of appearances per Age (or customer proxy)
df_mock["Frequency"] = df_mock.groupby("Age")["Title"].transform("count")

#### Monetary -> Average Price per Age (or customer proxy)
df_mock["Monetary"] = df_mock.groupby("Age")["Price"].transform("mean")

# --------------------------
# Assign RFM Scores
# --------------------------

#### Recency: higher review count = better (invert labels so higher → 3)
df_mock["R_score"] = pd.qcut(df_mock["Recency"].rank(method="first"), q=3, labels=[1, 2, 3]).astype(int)

#### Frequency: higher = better
df_mock["F_score"] = pd.qcut(df_mock["Frequency"].rank(method="first"), q=3, labels=[1, 2, 3]).astype(int)

#### Monetary: higher = better
df_mock["M_score"] = pd.qcut(df_mock["Monetary"].rank(method="first"), q=3, labels=[1, 2, 3]).astype(int)


# --------------------------
#  Final RFM Segment
# --------------------------

df_mock["RFM_Segment"] = df_mock["R_score"].astype(str) + df_mock["F_score"].astype(str) + df_mock["M_score"].astype(str)
df_mock["RFM_Score"] = df_mock[["R_score", "F_score", "M_score"]].sum(axis=1)

# --------------------------
#  Define Customer Levels
# --------------------------

def segment_me(df):
    if df["RFM_Score"] >= 8:
        return "High-Value"
    elif df["RFM_Score"] >= 5:
        return "Mid-Value"
    else:
        return "Low-Value"

df_mock["Customer_Segment"] = df_mock.apply(segment_me, axis=1)

# --------------------------
#  Output
# --------------------------

rfm_summary = df_mock.groupby("Customer_Segment")[["Recency", "Frequency", "Monetary"]].mean().round(2)

print("✅ RFM Segmentation Done!")
print(rfm_summary.head())

### Save cleaned RFM file
output_path = r"/Users/vini/Downloads/farfetch hackathon data/RFM_Output.csv"
df_mock.to_csv(output_path, index=False)
print(f"📂 File saved to: {output_path}")


df_mock["Monetary"] = df_mock.groupby("Age")["Price"].transform("mean")

# --------------------------
#  RFM Scoring
# --------------------------
df_mock["R_score"] = pd.qcut(df_mock["Recency"].rank(method="first"), q=3, labels=[1, 2, 3]).astype(int)
df_mock["F_score"] = pd.qcut(df_mock["Frequency"].rank(method="first"), q=3, labels=[1, 2, 3]).astype(int)
df_mock["M_score"] = pd.qcut(df_mock["Monetary"].rank(method="first"), q=3, labels=[1, 2, 3]).astype(int)

df_mock["RFM_Segment"] = df_mock["R_score"].astype(str) + df_mock["F_score"].astype(str) + df_mock["M_score"].astype(str)
df_mock["RFM_Score"] = df_mock[["R_score", "F_score", "M_score"]].sum(axis=1)

def segment_me(df):
    if df["RFM_Score"] >= 8:
        return "High-Value"
    elif df["RFM_Score"] >= 5:
        return "Mid-Value"
    else:
        return "Low-Value"

df_mock["Customer_Segment"] = df_mock.apply(segment_me, axis=1)


# --------------------------
# Summary
# --------------------------
rfm_summary = df_mock.groupby("Customer_Segment")[["Recency", "Frequency", "Monetary"]].mean().round(2)
print("✅ RFM Segmentation Done!")
print(rfm_summary)

# --------------------------
#  Visualizations
# --------------------------

### Bar plot of customer segments
plt.figure(figsize=(6,4))
sns.countplot(data=df_mock, x="Customer_Segment", palette="viridis", order=["High-Value", "Mid-Value", "Low-Value"])
plt.title("Customer Segment Distribution")
plt.xlabel("Segment")
plt.ylabel("Count")
plt.show()

### Boxplot for Recency, Frequency, Monetary by Segment
fig, axes = plt.subplots(1, 3, figsize=(18,5))

sns.boxplot(data=df_mock, x="Customer_Segment", y="Recency", order=["High-Value", "Mid-Value", "Low-Value"], palette="viridis", ax=axes[0])
axes[0].set_title("Recency by Segment")

sns.boxplot(data=df_mock, x="Customer_Segment", y="Frequency", order=["High-Value", "Mid-Value", "Low-Value"], palette="viridis", ax=axes[1])
axes[1].set_title("Frequency by Segment")

sns.boxplot(data=df_mock, x="Customer_Segment", y="Monetary", order=["High-Value", "Mid-Value", "Low-Value"], palette="viridis", ax=axes[2])
axes[2].set_title("Monetary by Segment")

plt.tight_layout()
plt.show()

### Heatmap for average RFM values
plt.figure(figsize=(6,4))
sns.heatmap(rfm_summary.T, annot=True, cmap="YlGnBu", cbar=False)
plt.title("Average RFM Values by Customer Segment")
plt.show()



# -----------------------------------
# Churn Analysis
# -----------------------------------


### -----------------------------------
### Define Churn vs Active
### -----------------------------------

#### Define thresholds
recency_threshold = df_mock["Recency"].median()
freq_threshold = df_mock["Frequency"].median()

def churn_flag(row):
    if row["Recency"] < recency_threshold and row["Frequency"] < freq_threshold:
        return "Churn"
    else:
        return "Active"

df_mock["Churn_Status"] = df_mock.apply(churn_flag, axis=1)


### Churn Rate

churn_rate = df_mock["Churn_Status"].value_counts(normalize=True) * 100
print("Churn Rate (%)\n", churn_rate)


## Compare Churn vs Active

#### By Age Group
df_mock["AgeGroup"] = pd.cut(
    df_mock["Age"],
    bins=[0, 24, 40, 55, 75, 100],
    labels=["Gen Z (<25)", "Millennials (25-40)", "Gen X (40-55)", "Boomers (55-75)", "Older (75+)"]
)

plt.figure(figsize=(8,5))
sns.countplot(data=df_mock, x="AgeGroup", hue="Churn_Status", palette="Set2")
plt.title("Churn vs Active by Age Group")
plt.xticks(rotation=45)
plt.show()

#### By Category (proxy for gender/region)
plt.figure(figsize=(10,5))
sns.countplot(data=df_mock, x="Category", hue="Churn_Status", palette="Set1",
              order=df_mock["Category"].value_counts().index)
plt.title("Churn vs Active by Category")
plt.xticks(rotation=45)
plt.show()

#### By Order Value
plt.figure(figsize=(6,5))
sns.boxplot(data=df_mock, x="Churn_Status", y="Monetary", palette="coolwarm")
plt.title("Churn vs Active by Order Value (AOV)")
plt.show()

#### By Rating
plt.figure(figsize=(6,5))
sns.boxplot(data=df_mock, x="Churn_Status", y="Rating", palette="viridis")
plt.title("Churn vs Active by Product Ratings")
plt.show()


# -----------------------------
# PRODUCT INSIGHTS - Sales
# -----------------------------

print("\n🔎 Product Insights")

### Best-selling product categories
if "Category" in df_mock.columns:
    best_categories = (
        df_mock.groupby("Category")["Frequency"].sum()
        .sort_values(ascending=False)
        .head(10)
    )
    
    
 print("\n📌 Top Categories by Sales:\n", best_categories)
    
    plt.figure(figsize=(8,5))
    sns.barplot(x=best_categories.values, y=best_categories.index, palette="mako")
    plt.title("Top Product Categories by Sales")
    plt.xlabel("Total Purchases (Frequency)")
    plt.ylabel("Category")
    plt.show()

### Price vs Popularity (scatter plot)
if "Price" in df_mock.columns:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df_mock, x="Price", y="Frequency", alpha=0.6)
    plt.title("Price vs Popularity (Frequency)")
    plt.xlabel("Price")
    plt.ylabel("Popularity (Purchases/Reviews)")
    plt.show()


### Top brands/products by revenue
if "Brand" in df_mock.columns and "Monetary" in df_mock.columns:
    top_brands = (
        df_mock.groupby("Brand")["Monetary"].sum()
        .sort_values(ascending=False)
        .head(10)
    )
    print("\n💎 Top Brands by Revenue:\n", top_brands)
    
    plt.figure(figsize=(8,5))
    sns.barplot(x=top_brands.values, y=top_brands.index, palette="viridis")
    plt.title("Top 10 Brands by Revenue")
    plt.xlabel("Revenue")
    plt.ylabel("Brand")
    plt.show()

    


#%%
# -----------------------------
# PRODUCT CLUSTERING - Mock
# -----------------------------

#### clustering_only.py

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans



## Feature Engineering


df_mock["LogPrice"] = np.log1p(df_mock["Price"])  # log transform for skew
df_mock["Review_Count"] = pd.to_numeric(df_mock["Review Count"], errors="coerce").fillna(0)

### Encode feedback sentiment
df_mock["Feedback_Score"] = df_mock["feedback"].astype(str).str.lower().map({
    "positive": 2,
    "neutral": 1,
    "negative": 0
})
df_mock["Feedback_Score"] = df_mock["Feedback_Score"].fillna(1)  # default neutral

### Select features for clustering
features = ["LogPrice", "Review_Count", "Age", "Feedback_Score"]
df_cluster = df_mock[features].dropna().copy()


## Scaling & KMeans


scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster)

kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df_cluster["Cluster"] = kmeans.fit_predict(X_scaled)


## Visualization


plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=df_cluster,
    x="LogPrice",
    y="Review_Count",
    hue="Cluster",
    palette="Set2",
    s=60
)
plt.title("Product Clusters")
plt.xlabel("Log Price")
plt.ylabel("Review Count")
plt.legend(title="Cluster")
plt.show()

print("✅ Clustering done. Cluster counts:")
print(df_cluster["Cluster"].value_counts())

## Cluster profiling
cluster_profile = df_cluster.groupby("Cluster").mean()
print("\n📊 Cluster Profiles:\n", cluster_profile)

sse = []
for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(6,4))
plt.plot(range(2, 8), sse, marker="o")
plt.title("Elbow Method for Optimal k")
plt.xlabel("Number of clusters")
plt.ylabel("SSE (Inertia)")
plt.show()


# -----------------------------
# Business Insight - Sales
# ----------------------------- 

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans



### Select features for clustering (adjust these to what you actually have)
features = ["Recency", "Frequency", "Monetary"]  # Example RFM columns
X = df_mock[features].copy()

### Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Apply KMeans
kmeans = KMeans(n_clusters=4, random_state=42)
df_mock["Cluster"] = kmeans.fit_predict(X_scaled)

### Now df_mock DEFINITELY has Cluster column
print(df_mock.head())

### Group by Cluster - Revenue (if you have Price column)
if "Price" in df_mock.columns:
    cluster_revenue = df_mock.groupby("Cluster")["Price"].sum().sort_values(ascending=False)
    print("\nRevenue by Cluster:\n", cluster_revenue)

### Group by Cluster - Size of each cluster
cluster_size = df_mock["Cluster"].value_counts().sort_index()
print("\nCluster Sizes:\n", cluster_size)

### Export for Tableau
output_path = "/Users/vini/Downloads/farfetch hackathon data/clustered_output.xlsx"
df_mock.to_excel(output_path, index=False)
print(f"\nClustered data saved to: {output_path}")

